# ResNet-50 with Neutrino Dataset

### Importing essential libraries
this will be cleaned, some are inessential

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

In [ ]:
import sklearn
from torchvision.models.resnet import ResNet, BasicBlock, Bottleneck
from torchvision.datasets import MNIST
from tqdm.autonotebook import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import inspect
import time
from torch import nn, optim
import torch
from torchvision.transforms import Compose, ToTensor, Normalize, Resize
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import numpy as np

I check here if cuda gpu is available

In [ ]:
torch.cuda.is_available()

## Model, Train, and Test

I define here the model (ResNet-50) and the train and test function. The model takes as input images channel first with 3 channels. The test definition is week and it could lead to a memory leak. 

In [ ]:
#pred_list and target_list are two lists of cuda tensors containing respectively the predictions and the 
#target values. Each tensor will be test_batch_size in lenght. Each list will contain 
#(amount of test samples/test_batch_size)*epochs samples. E.g. with 10k test samples, test_batch_size = 1k and
#4 epochs you'll get a list of 40 elements, each element 1k. Further down these lists will be converted in arrays.

pred_list = []
target_list = []
accuracy = []
loss = []
accuracy_train = []
loss_train = []
epochs = 10
test_batch = 100
    
class MnistResNet(ResNet):
    def __init__(self):
        super(MnistResNet, self).__init__(Bottleneck, [3,4,6,3], num_classes=3)
        self.conv1 = torch.nn.Conv2d(3, 64,
            kernel_size=(7, 7), 
            stride=(2, 2), 
            padding=(3, 3), bias=False)
######if you then use nll_loss you have to use log_sotmax        
    def forward(self, x):
        return F.log_softmax(
            super(MnistResNet, self).forward(x), dim=-1)

        



def train(args, model, device, train_loader, optimizer, epoch):
    train_loss = 0   
    correct = 0
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        train_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        output = None
        del output
        target = None
        del target
        length = len(data)
        data = None
        del data
        pred = None
        del pred
        
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * length, len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            
    train_loss /= len(train_loader.dataset)
    print('\nTrain set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        train_loss, correct, len(train_loader.dataset),
        100. * correct / len(train_loader.dataset)))
    accuracy_train.append(100. * correct / len(train_loader.dataset))
    loss_train.append(train_loss)


def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            data = None
            del data
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
            pred_list.append(pred)
            target_list.append(target)
            output = None
            del output
            pred = None
            del pred
            
            
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    accuracy.append(100. * correct / len(test_loader.dataset))
    loss.append(test_loss)

## Dataset

I implemented here a simple dataset. At the moment, it takes a tensor as data and numpy array as target. Moreover, it only takes one file. This is something that I will need to change in the future since we have several files.

In [ ]:
#implementing DataSet

class NeutrinoDataset(Dataset):
    """The training table dataset.
    """
    def __init__(self, x_tensor, y_tensor):
        #x_filenames = glob(x_path + '*.png') # Get the filenames of all training images
        
        #self.x_data = [torch.from_numpy(misc.imread(filename)) for filename in x_filenames] # Load the images into torch tensors
        #self.y_data = target_label_list # Class labels
        self.x_data = x_tensor
        self.y_data = y_tensor
        self.len = len(self.x_data)
        
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
        
    def __len__(self):
        return self.len

In [ ]:
x_train_temp_0 = np.load('/home/stefano/neutrino/data/final_array_x/train/x_train_numu_500_0.npy')
y_train_0 = np.load('/home/stefano/neutrino/data/final_array_y/train/y_train_numu_500_event_0.npy')

x_train_temp_1 = np.load('/home/stefano/neutrino/data/final_array_x/train/x_train_numu_500_1.npy')
y_train_1 = np.load('/home/stefano/neutrino/data/final_array_y/train/y_train_numu_500_event_1.npy')

x_train_temp_2 = np.load('/home/stefano/neutrino/data/final_array_x/train/x_train_numu_500_2.npy')
y_train_2 = np.load('/home/stefano/neutrino/data/final_array_y/train/y_train_numu_500_event_2.npy')

x_train_numu_3 = np.load('/home/stefano/neutrino/data/final_array_x/train/x_train_numu_500_3.npy')
y_train_numu_3 = np.load('/home/stefano/neutrino/data/final_array_y/train/y_train_numu_500_event_3.npy')

x_train_numu_4 = np.load('/home/stefano/neutrino/data/final_array_x/train/x_train_numu_500_4.npy')
y_train_numu_4 = np.load('/home/stefano/neutrino/data/final_array_y/train/y_train_numu_500_event_4.npy')

x_train_numu_5 = np.load('/home/stefano/neutrino/data/final_array_x/train/x_train_numu_500_5.npy')
y_train_numu_5 = np.load('/home/stefano/neutrino/data/final_array_y/train/y_train_numu_500_event_5.npy')

x_train_numu_6 = np.load('/home/stefano/neutrino/data/final_array_x/train/x_train_numu_500_6.npy')
y_train_numu_6 = np.load('/home/stefano/neutrino/data/final_array_y/train/y_train_numu_500_event_6.npy')

x_train_numu_7 = np.load('/home/stefano/neutrino/data/final_array_x/train/x_train_numu_500_7.npy')
y_train_numu_7 = np.load('/home/stefano/neutrino/data/final_array_y/train/y_train_numu_500_event_7.npy')

x_train_numu_8 = np.load('/home/stefano/neutrino/data/final_array_x/train/x_train_numu_500_8.npy')
y_train_numu_8 = np.load('/home/stefano/neutrino/data/final_array_y/train/y_train_numu_500_event_8.npy')

x_train_numu_9 = np.load('/home/stefano/neutrino/data/final_array_x/train/x_train_numu_500_9.npy')
y_train_numu_9 = np.load('/home/stefano/neutrino/data/final_array_y/train/y_train_numu_500_event_9.npy')

x_train_temp_3 = np.load('/home/stefano/neutrino/data/final_array_x/train/x_train_nue_500_0.npy')
y_train_3 = np.load('/home/stefano/neutrino/data/final_array_y/train/y_train_nue_500_event_0.npy')

x_train_temp_4 = np.load('/home/stefano/neutrino/data/final_array_x/train/x_train_nue_500_1.npy')
y_train_4 = np.load('/home/stefano/neutrino/data/final_array_y/train/y_train_nue_500_event_1.npy')

x_train_temp_5 = np.load('/home/stefano/neutrino/data/final_array_x/train/x_train_nue_500_2.npy')
y_train_5 = np.load('/home/stefano/neutrino/data/final_array_y/train/y_train_nue_500_event_2.npy')

x_train_temp_6 = np.load('/home/stefano/neutrino/data/final_array_x/train/x_train_nue_500_3.npy')
y_train_6 = np.load('/home/stefano/neutrino/data/final_array_y/train/y_train_nue_500_event_3.npy')

x_train_nue_4 = np.load('/home/stefano/neutrino/data/final_array_x/train/x_train_nue_500_4.npy')
y_train_nue_4 = np.load('/home/stefano/neutrino/data/final_array_y/train/y_train_nue_500_event_4.npy')

x_train_nue_5 = np.load('/home/stefano/neutrino/data/final_array_x/train/x_train_nue_500_5.npy')
y_train_nue_5 = np.load('/home/stefano/neutrino/data/final_array_y/train/y_train_nue_500_event_5.npy')

x_train_nue_6 = np.load('/home/stefano/neutrino/data/final_array_x/train/x_train_nue_500_6.npy')
y_train_nue_6 = np.load('/home/stefano/neutrino/data/final_array_y/train/y_train_nue_500_event_6.npy')

x_train_nue_7 = np.load('/home/stefano/neutrino/data/final_array_x/train/x_train_nue_500_7.npy')
y_train_nue_7 = np.load('/home/stefano/neutrino/data/final_array_y/train/y_train_nue_500_event_7.npy')

x_train_nue_8 = np.load('/home/stefano/neutrino/data/final_array_x/train/x_train_nue_500_8.npy')
y_train_nue_8 = np.load('/home/stefano/neutrino/data/final_array_y/train/y_train_nue_500_event_8.npy')

x_train_nue_9 = np.load('/home/stefano/neutrino/data/final_array_x/train/x_train_nue_500_9.npy')
y_train_nue_9 = np.load('/home/stefano/neutrino/data/final_array_y/train/y_train_nue_500_event_9.npy')
                         
x_test_numu_0 = np.load('/home/stefano/neutrino/data/final_array_x/test/x_test_numu_500_0.npy')
y_test_numu_0 = np.load('/home/stefano/neutrino/data/final_array_y/test/y_test_numu_500_event_0.npy')

x_test_numu_1 = np.load('/home/stefano/neutrino/data/final_array_x/test/x_test_numu_500_1.npy')
y_test_numu_1 = np.load('/home/stefano/neutrino/data/final_array_y/test/y_test_numu_500_event_1.npy')

x_test_nue_0 = np.load('/home/stefano/neutrino/data/final_array_x/test/x_test_nue_500_0.npy')
y_test_nue_0 = np.load('/home/stefano/neutrino/data/final_array_y/test/y_test_nue_500_event_0.npy')

x_test_nue_1 = np.load('/home/stefano/neutrino/data/final_array_x/test/x_test_nue_500_1.npy')
y_test_nue_1 = np.load('/home/stefano/neutrino/data/final_array_y/test/y_test_nue_500_event_1.npy')
    

#concatenate all the files together and then delete from the memory unnecessary files
x_train_temp_a=np.concatenate((x_train_temp_0, x_train_temp_1))
x_train_temp_b=np.concatenate((x_train_temp_a, x_train_temp_2))
x_train_temp_0 = None
del x_train_temp_0
x_train_temp_1 = None
del x_train_temp_1
x_train_temp_2 = None
del x_train_temp_2
x_train_temp_a = None
del x_train_temp_a
x_train_temp_c=np.concatenate((x_train_temp_b, x_train_temp_3))
x_train_temp_b=None
del x_train_temp_b
x_train_temp_d=np.concatenate((x_train_temp_c, x_train_temp_4))
x_train_temp_c=None
del x_train_temp_c
x_train_temp_e=np.concatenate((x_train_temp_d, x_train_temp_5))
x_train_temp_3 = None
del x_train_temp_3
x_train_temp_4 = None
del x_train_temp_4
x_train_temp_5 = None
del x_train_temp_5
x_train_temp_d = None
del x_train_temp_d
x_train_temp_f=np.concatenate((x_train_temp_e, x_train_temp_6))
x_train_temp_e=None
del x_train_temp_e
x_train_temp_6 = None
del x_train_temp_6
x_train_temp_g=np.concatenate((x_train_temp_f, x_train_numu_3))
x_train_temp_f=None
del x_train_temp_f
x_train_numu_3 = None
del x_train_numu_3
x_train_temp_h=np.concatenate((x_train_temp_g, x_train_nue_4))
x_train_temp_g=None
del x_train_temp_g
x_train_nue_4 = None
del x_train_nue_4
x_train_temp_i=np.concatenate((x_train_temp_h, x_train_numu_4))
x_train_temp_h=None
del x_train_temp_h
x_train_numu_4 = None
del x_train_numu_4
x_train_temp_l=np.concatenate((x_train_temp_i, x_train_nue_5))
x_train_temp_i=None
del x_train_temp_i
x_train_nue_5 = None
del x_train_nue_5
x_train_temp_m=np.concatenate((x_train_temp_l, x_train_numu_5))
x_train_temp_l=None
del x_train_temp_l
x_train_numu_5 = None
del x_train_numu_5
x_train_temp_n=np.concatenate((x_train_temp_m, x_train_nue_6))
x_train_temp_m=None
del x_train_temp_m
x_train_nue_6 = None
del x_train_nue_6
x_train_temp_o=np.concatenate((x_train_temp_n, x_train_numu_6))
x_train_temp_n=None
del x_train_temp_n
x_train_numu_6 = None
del x_train_numu_6
x_train_temp_p=np.concatenate((x_train_temp_o, x_train_nue_7))
x_train_temp_o=None
del x_train_temp_o
x_train_nue_7 = None
del x_train_nue_7
x_train_temp_q=np.concatenate((x_train_temp_p, x_train_numu_7))
x_train_temp_p=None
del x_train_temp_p
x_train_numu_7 = None
del x_train_numu_7
x_train_temp_r=np.concatenate((x_train_temp_q, x_train_nue_8))
x_train_temp_q=None
del x_train_temp_q
x_train_nue_8 = None
del x_train_nue_8
x_train_temp_s=np.concatenate((x_train_temp_r, x_train_numu_8))
x_train_temp_r=None
del x_train_temp_r
x_train_numu_8 = None
del x_train_numu_8
x_train_temp_t=np.concatenate((x_train_temp_s, x_train_nue_9))
x_train_temp_s=None
del x_train_temp_s
x_train_nue_9 = None
del x_train_nue_9
x_train_temp=np.concatenate((x_train_temp_t, x_train_numu_9))
x_train_temp_t=None
del x_train_temp_t
x_train_numu_9 = None
del x_train_numu_9



x_test_temp_a=np.concatenate((x_test_numu_0, x_test_nue_0))
x_test_numu_0 = None
del x_test_numu_0
x_test_nue_0 = None
del x_test_nue_0
x_test_temp_b=np.concatenate((x_test_temp_a, x_test_nue_1))
x_test_temp_a = None
del x_test_temp_a
x_test_nue_1 = None
del x_test_nue_1
x_test_temp=np.concatenate((x_test_temp_b, x_test_numu_1))
x_test_temp_b = None
del x_test_temp_b
x_test_numu_1 = None
del x_test_numu_1

y_train_a=np.concatenate((y_train_0, y_train_1))
y_train_0 = None
del y_train_0
y_train_1 = None
del y_train_1
y_train_b=np.concatenate((y_train_a, y_train_2))
y_train_c=np.concatenate((y_train_b, y_train_3))
y_train_a=None
del y_train_a
y_train_b=None
del y_train_b
y_train_d=np.concatenate((y_train_c, y_train_4))
y_train_c=None
del y_train_c
y_train_e=np.concatenate((y_train_d, y_train_5))
y_train_2 = None
del y_train_2
y_train_3 = None
del y_train_3
y_train_4 = None
del y_train_4
y_train_5 = None
del y_train_5
y_train_d=None
del y_train_d
y_train_f=np.concatenate((y_train_e, y_train_6))
y_train_e=None
del y_train_e
y_train_6 = None
del y_train_6
y_train_g=np.concatenate((y_train_f, y_train_numu_3))
y_train_f=None
del y_train_f
y_train_numu_3 = None
del y_train_numu_3
y_train_h=np.concatenate((y_train_g, y_train_nue_4))
y_train_g=None
del y_train_g
y_train_nue_4 = None
del y_train_nue_4
y_train_i=np.concatenate((y_train_h, y_train_numu_4))
y_train_h=None
del y_train_h
y_train_numu_4 = None
del y_train_numu_4
y_train_l=np.concatenate((y_train_i, y_train_nue_5))
y_train_i=None
del y_train_i
y_train_nue_5 = None
del y_train_nue_5
y_train_m=np.concatenate((y_train_l, y_train_numu_5))
y_train_l=None
del y_train_l
y_train_numu_5 = None
del y_train_numu_5
y_train_n=np.concatenate((y_train_m, y_train_nue_6))
y_train_m=None
del y_train_m
y_train_nue_6 = None
del y_train_nue_6
y_train_o=np.concatenate((y_train_n, y_train_numu_6))
y_train_n=None
del y_train_n
y_train_numu_6 = None
del y_train_numu_6
y_train_p=np.concatenate((y_train_o, y_train_nue_7))
y_train_o=None
del y_train_o
y_train_nue_7 = None
del y_train_nue_7
y_train_q=np.concatenate((y_train_p, y_train_numu_7))
y_train_p=None
del y_train_p
y_train_numu_7 = None
del y_train_numu_7
y_train_r=np.concatenate((y_train_q, y_train_nue_8))
y_train_q=None
del y_train_q
y_train_nue_8 = None
del y_train_nue_8
y_train_s=np.concatenate((y_train_r, y_train_numu_8))
y_train_r=None
del y_train_r
y_train_numu_8 = None
del y_train_numu_8
y_train_t=np.concatenate((y_train_s, y_train_nue_9))
y_train_s=None
del y_train_s
y_train_nue_9 = None
del y_train_nue_9
y_train=np.concatenate((y_train_t, y_train_numu_9))
y_train_t=None
del y_train_t
y_train_numu_9 = None
del y_train_numu_9


y_test_a=np.concatenate((y_test_numu_0, y_test_nue_0))
y_test_numu_0 = None
del y_test_numu_0
y_test_nue_0 = None
del y_test_nue_0
y_test_b=np.concatenate((y_test_a, y_test_nue_1))
y_test_a = None
del y_test_a
y_test_nue_1 = None
del y_test_nue_1
y_test=np.concatenate((y_test_b, y_test_numu_1))
y_test_b = None
del y_test_b
y_test_numu_1 = None
del y_test_numu_1

print('x_train_temp.shape',x_train_temp.shape)
print('x_test_temp.shape',x_test_temp.shape)
print('y_train.shape',y_train.shape)
print('y_test.shape',y_test.shape)


## Importing files and preparing the dataset

In this part I import the train and test files, I normalize them and I reshape them to be channel first. This part could also lead to potential memory leaks. It would be nice to convert and normalise the numpy array in one go, but at the moment I can't.

In [ ]:
###this method is broken!!! do not use until fixed

import os

folder_x_train='/home/stefano/neutrino/data/final_array_x/train/'
directory_x_train=os.fsencode(folder_x_train)

folder_x_test='/home/stefano/neutrino/data/final_array_x/test/'
directory_x_test=os.fsencode(folder_x_test)

folder_y_train='/home/stefano/neutrino/data/final_array_y/train/'
directory_y_train=os.fsencode(folder_y_train)

folder_y_test='/home/stefano/neutrino/data/final_array_y/test/'
directory_y_test=os.fsencode(folder_y_test)

counter_a=0
counter_b=0
counter_c=0
counter_d=0

for file in os.listdir(directory_x_train):
    x_train=np.load(folder_x_train+os.fsdecode(file))
    if counter_a==0:
        x_train_temp=x_train
    elif counter_a>0:
        x_train_temp=np.concatenate((x_train_temp,x_train))
        
    counter_a=counter_a+1
    del x_train
        
for file in os.listdir(directory_x_test):
    x_test=np.load(folder_x_test+os.fsdecode(file))
    if counter_b==0:
        x_test_temp=x_test
    elif counter_b>0:
        x_test_temp=np.concatenate((x_test_temp,x_test))
        
    counter_b=counter_b+1
    del x_test

for file in os.listdir(directory_y_train):
    y_tr=np.load(folder_y_train+os.fsdecode(file))
    if counter_c==0:
        y_train=y_tr
    elif counter_c>0:
        y_train=np.concatenate((y_train,y_tr))
        
    counter_c=counter_c+1
    del y_tr
    
for file in os.listdir(directory_y_test):
    y_ts=np.load(folder_y_test+os.fsdecode(file))
    if counter_d==0:
        y_test=y_ts
    elif counter_d>0:
        y_test=np.concatenate((y_test,y_ts))
        
    counter_d=counter_d+1
    del y_ts
    
print(x_train_temp.shape)
print(x_test_temp.shape)
print(y_train.shape)
print(y_test.shape)

del counter_a
del counter_b
del counter_c
del counter_d

In [ ]:
#passages are: x_train_temp -> x_train (min 0 max 1, float32) -> x_train_reshaped (channel first) -> x_train_tensor

#normalise the data
old_min = np.min(x_train_temp)
old_max = np.max(x_train_temp)
print("Before:", old_min, old_max)
x_train = x_train_temp.astype('float32')
x_test = x_test_temp.astype('float32')
x_train /=255
x_test /= 255
new_min = np.min(x_train)
new_max = np.max(x_train)
print("After:", new_min, new_max)

print("Test Data Shape after conversion to float")
    
    #read the dimensions from one example in the trainig set
img_rows, img_cols = x_train[0].shape[0], x_train[0].shape[1]
print(x_train[0].shape[0])
print(x_train[0].shape[1])

#read the dimensions from one example in the trainig set
img_rows, img_cols = x_train[0].shape[0], x_train[0].shape[1]

#Different NN libraries (e.g., TF) use different ordering of dimensions
#Here we set the "input shape" so that later the NN knows what shape to expect
x_train_reshaped = x_train.reshape(x_train.shape[0], 3, img_rows, img_cols)
x_test_reshaped = x_test.reshape(x_test.shape[0], 3, img_rows, img_cols)
input_shape = (3, img_rows, img_cols)

print("input_shape",input_shape)
print("x_train[0].shape[0]", x_train_reshaped[0].shape[0])
print("x_train[0].shape[1]", x_train_reshaped[0].shape[1])
print("x_train[0].shape[2]", x_train_reshaped[0].shape[2])
print("x_train.shape",x_train.shape)

x_train_tensor = torch.from_numpy(x_train_reshaped).float()
x_test_tensor = torch.from_numpy(x_test_reshaped).float()

x_train = None
del x_train
x_test = None
del x_test
x_train_reshaped = None
del x_train_reshaped
x_test_reshaped = None
del x_test_reshaped

y_train_tensor=torch.from_numpy(y_train)
y_test_tensor=torch.from_numpy(y_test)

#y_train = None
#del y_train
#y_test = None
#del y_test

dataset_test = NeutrinoDataset(x_test_tensor, y_test_tensor)

## Definition of main function

I define here the dataset and the main function. This seems to be the stronger part of the code.

In [ ]:
import easydict
def main():
    args = easydict.EasyDict({
        "batch_size": 16,#it was 64
        "test_batch_size": test_batch,
        "epochs": epochs,
        "lr": 1.0,
        "gamma": 0.7,
        "no_cuda": False,
        "seed": 1,
        "log_interval":10,
        "save_model": False
    })
    use_cuda = not args.no_cuda and torch.cuda.is_available()

    torch.manual_seed(args.seed)

    device = torch.device("cuda" if use_cuda else "cpu")
    #device = torch.device("cpu")
    
    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
    
    dataset_train = NeutrinoDataset(x_train_tensor, y_train_tensor)
    dataset_test = NeutrinoDataset(x_test_tensor, y_test_tensor)
    
    train_loader = torch.utils.data.DataLoader(dataset=dataset_train,
        batch_size=args.batch_size, shuffle=True, **kwargs)
    test_loader = torch.utils.data.DataLoader(dataset_test,
        batch_size=args.test_batch_size, shuffle=False, **kwargs)

    model = MnistResNet().to(device)
    optimizer = optim.Adadelta(model.parameters(), lr=args.lr)

    scheduler = StepLR(optimizer, step_size=1, gamma=args.gamma)
    for epoch in range(1, args.epochs + 1):
        train(args, model, device, train_loader, optimizer, epoch)
        test(args, model, device, test_loader)
        scheduler.step()

    if args.save_model:
        torch.save(model.state_dict(), "mnist_cnn.pt")


### Plotting your Train and Test Dataset
It is important to always check your dataset. With the following lines images from your train and test dataset and randomly selected and plotted.

In this part I calcolate mean and std of the pics in case I find a way to use tensor.normalise. Skip this part when run the code.

## Visualisation

The following two cells contain a cool tool to visualise train and test dataset. Useful.

In [ ]:
use_cuda =  torch.cuda.is_available()

device = torch.device("cuda" if use_cuda else "cpu")
import zlib
import matplotlib
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter("ignore", DeprecationWarning)
figsize = [14, 5]
starting_number = torch.IntTensor.item(torch.LongTensor(1).random_(0, len(x_train_temp)-1))#-(nrows*ncols)))

print(x_train_temp.shape)
#if use_cuda == True:
#    x_train_cpu=x_train_temp.cpu()
#    y_train_cpu=y_train_tensor.cpu()
#else:
#    x_train_cpu=x_train_temp
#    y_train_cpu=y_train_tensor
    
print("now showing image ",(starting_number))

####this part is for the true value
#0 is for NC events
#1 is for numu CC events
#2 is for nue CC events
truth=y_train[starting_number]
if truth==0:
    truth_stg="NC"
elif truth==1:
    truth_stg="numu CC"
elif truth==2:
    truth_stg="nue CC"

a = x_train_temp[(starting_number),:,:,0]

b = x_train_temp[(starting_number),:,:,1]

c = x_train_temp[(starting_number),:,:,2]


f, axarr = plt.subplots(1,3,figsize=figsize)#, constrained_layout=True)
f.suptitle('Event Seen From Wire Planes - 200 x 200 pixels - this is a '+truth_stg+' event')
im=axarr[0].imshow(a, norm=matplotlib.colors.LogNorm())
im2=axarr[1].imshow(b, norm=matplotlib.colors.LogNorm())
im3=axarr[2].imshow(c, norm=matplotlib.colors.LogNorm())
axarr[0].set_title('U Plane')
axarr[1].set_title('V Plane')
axarr[2].set_title('W Plane')
bar1=f.colorbar(im,ax=axarr[0],fraction=0.046,pad=0.04)
bar2=f.colorbar(im2,ax=axarr[1],fraction=0.046,pad=0.04)
bar3=f.colorbar(im3,ax=axarr[2],fraction=0.046,pad=0.04)
bar1.set_label('Energy [MeV]')
bar2.set_label('Energy [MeV]')
bar3.set_label('Energy [MeV]')
f.tight_layout() #this is to add more space between plots
for ax in axarr.flat:
    ax.set(xlabel='Wire Plane', ylabel='Time')

plt.show()
plt.close()


In [ ]:
figsize = [14, 5]
starting_number = torch.IntTensor.item(torch.LongTensor(1).random_(0, len(x_test_temp)-1))#-(nrows*ncols)))
    
print("now showing image ",(starting_number))

####this part is for the true value
#0 is for NC events
#1 is for numu CC events
#2 is for nue CC events
truth=y_test[starting_number]
if truth==0:
    truth_stg="NC"
elif truth==1:
    truth_stg="numu CC"
elif truth==2:
    truth_stg="nue CC"

a = x_test_temp[(starting_number),:,:,0]

b = x_test_temp[(starting_number),:,:,1]

c = x_test_temp[(starting_number),:,:,2]


f, axarr = plt.subplots(1,3,figsize=figsize)#, constrained_layout=True)
f.suptitle('Event Seen From Wire Planes - 200 x 200 pixels - this is a '+truth_stg+' event')
im=axarr[0].imshow(a, norm=matplotlib.colors.LogNorm())
im2=axarr[1].imshow(b, norm=matplotlib.colors.LogNorm())
im3=axarr[2].imshow(c, norm=matplotlib.colors.LogNorm())
axarr[0].set_title('U Plane')
axarr[1].set_title('V Plane')
axarr[2].set_title('W Plane')
bar1=f.colorbar(im,ax=axarr[0],fraction=0.046,pad=0.04)
bar2=f.colorbar(im2,ax=axarr[1],fraction=0.046,pad=0.04)
bar3=f.colorbar(im3,ax=axarr[2],fraction=0.046,pad=0.04)
bar1.set_label('Energy [MeV]')
bar2.set_label('Energy [MeV]')
bar3.set_label('Energy [MeV]')
f.tight_layout() #this is to add more space between plots
for ax in axarr.flat:
    ax.set(xlabel='Wire Plane', ylabel='Time')

plt.show()
plt.close()

In [ ]:
start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)

## Main Run


In [ ]:
start.record()
main()
end.record()

In [ ]:
print(start.elapsed_time(end))

In [ ]:
#resnet50 size 200x200 with cpu (25-02-2020) batch size 1
# with cpu is was 3717510.25 ms, that is ~3717 s ~ 1 h for 6 epochs - final test set batch size 1
# Test set: Average loss: 3.5087, Accuracy: 487/1000 (49%)

#resnet50 size 200x200 with gpu (25-02-2020) batch size 1
# with gpu is was 381100.1875 ms, that is ~381 s ~ 6 and a half min for 6 epochs - final test set batch size 1
# Test set: Average loss: 2.9999, Accuracy: 499/1000 (50%)

#resnet50 size 200x200 with gpu (25-02-2020) batch size 16
# with gpu is was 329710.28125 ms, that is ~329 s ~ 5 min for 20 epochs - final test set batch size 100
# Test set: Average loss: 1.2409, Accuracy: 664/1000 (66%)

#resnet 50 11-03-2020
# 10 epochs time 1636127.375
# Test set: Average loss: 0.6086, Accuracy: 1553/2000 (78%)

In [ ]:
epochs_array = []
for i in range (1,epochs+1):
    epochs_array.append(i)
    
accuracy=np.array(accuracy)
accuracy=accuracy/100
accuracy_train=np.array(accuracy_train)
accuracy_train=accuracy_train/100
epochs_array = np.array(epochs_array)
loss = np.array(loss)
loss_train = np.array(loss_train)
plt.figure(figsize = (10,7))
plt.subplot(1, 2, 1)
plt.title('Accuracy vs. Epoch')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.plot(epochs_array,accuracy_train,color='C1',label='Training')
plt.plot(epochs_array,accuracy,label='Validation')
plt.legend()
plt.subplot(1, 2, 2)
plt.title('Total Loss vs. Epoch')
plt.ylabel('Total Loss')
plt.xlabel('Epoch')
plt.plot(epochs_array,loss_train,color='C1',label='Training')
plt.plot(epochs_array,loss,label='Validation')
plt.legend()
plt.tight_layout()

plt.show()

In [ ]:
pred_array_total = [] # array containing all predictions for all epochs
pred_array = [] #array containing only predictions of last epoch
for i in range(len(pred_list)):
    pred_array_total.append(pred_list[i].cpu().numpy())
    
pred_array_total=np.array(pred_array_total)
pred_array_total=np.squeeze(np.concatenate(pred_array_total))
#print(epochs*test_batch)
for i in range ((epochs*(2000)-2000),(epochs*2000)):
    pred_array.append(pred_array_total[i])
    
pred_array=np.array(pred_array)
pred_array.shape

In [ ]:
target_array_total = []#array containing all the targets for all epochs
target_array = []#array containing targets for last epoch
for i in range(len(target_list)):
    target_array_total.append(target_list[i].cpu().numpy())
    
target_array_total=np.array(target_array_total)
target_array_total=np.squeeze(np.concatenate(target_array_total))

for i in range ((epochs*2000)-2000,epochs*2000):
    target_array.append(target_array_total[i])
    
target_array=np.array(target_array)
target_array.shape

In [ ]:
import random
wrong_list = []
for i in range(len(target_array)):
    if target_array[i]!=pred_array[i]:
        wrong_list.append(i)
        
element=random.choice(wrong_list)

####this part is for the true value
#0 is for NC events
#1 is for numu CC events
#2 is for nue CC events
truth_check=y_test[element]
if truth_check==0:
    truth_stg="NC"
elif truth_check==1:
    truth_stg="numu CC"
elif truth_check==2:
    truth_stg="nue CC"
    
if pred_array[element]==0:
    pred_stg="NC"
elif pred_array[element]==1:
    pred_stg="numu CC"
elif pred_array[element]==2:
    pred_stg="nue CC"

#print('This was supposed to be ',target_array[element],' but it was predicted as ',pred_array[element])
print('This was supposed to be ',truth_stg,' but it was predicted as ',pred_stg)

a = x_test_temp[(element),:,:,0]

b = x_test_temp[(element),:,:,1]

c = x_test_temp[(element),:,:,2]


f, axarr = plt.subplots(1,3,figsize=figsize)#, constrained_layout=True)
f.suptitle('Event Seen From Wire Planes - 200 x 200 pixels - this is a '+truth_stg+' event')
im=axarr[0].imshow(a, norm=matplotlib.colors.LogNorm())
im2=axarr[1].imshow(b, norm=matplotlib.colors.LogNorm())
im3=axarr[2].imshow(c, norm=matplotlib.colors.LogNorm())
axarr[0].set_title('U Plane')
axarr[1].set_title('V Plane')
axarr[2].set_title('W Plane')
bar1=f.colorbar(im,ax=axarr[0],fraction=0.046,pad=0.04)
bar2=f.colorbar(im2,ax=axarr[1],fraction=0.046,pad=0.04)
bar3=f.colorbar(im3,ax=axarr[2],fraction=0.046,pad=0.04)
bar1.set_label('Energy [MeV]')
bar2.set_label('Energy [MeV]')
bar3.set_label('Energy [MeV]')
f.tight_layout() #this is to add more space between plots
for ax in axarr.flat:
    ax.set(xlabel='Wire Plane', ylabel='Time')

In [ ]:
from __future__ import print_function
torch.set_printoptions(linewidth = 120)
import sklearn
#print(sklearn.__version__)
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import plot_confusion_matrix


plt.figure(figsize=(50,50))
cm=sklearn.metrics.confusion_matrix(target_array, pred_array)#,normalize='all')#, classes=label_dict,
                     #title='Confusion matrix')
cm
#plt.show()

In [ ]:
import seaborn as sn
import pandas as pd
df_cm = pd.DataFrame(cm, index = [i for i in "012"],
                  columns = [i for i in "012"])
plt.figure(figsize = (14,10))
sn.heatmap(df_cm, annot=True)